In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from os import walk
import re
from scipy.signal import savgol_filter
from scipy.signal import find_peaks, find_peaks_cwt
from scipy import signal

import matplotlib.gridspec as gridspec
import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')

from _Methods.Methods import * # collection of processing methods in _Methods directory

data_dir = 'Messungen'

dic = getFileContent(data_dir)

In [ ]:
print('Following files have been found:')
for i in dic.keys():
    print(dic[i])

In [ ]:
# split data for cyclical analysis and ripping force analysis
# read all csv data into one list 
datasets = {}#filename = []

#renaming those absolutely horribly chosen column names
names = ['Points','Elapsed_Time_Sec','Scan_Time_Sec','Disp_mm','Load_2_N','E12_%','E22_%','Axial_cmd_V','Axial_err_%','Unnamed']

for key in dic.keys():
    #print(key)
    len_tmp = pd.read_csv(dic[key], index_col = False, names=names).shape[0]
    if len_tmp >2:
        datasets[key] = pd.read_csv(dic[key],header = 2, index_col = False, names=names)
        datasets[key]['Short'] = 0
        #if re.search(r"reis", key) == 1:
        #    print('RISS')
        #print(datasets[key].head())
    else:
        #print('tiny data set')
        datasets[key] = pd.read_csv(dic[key], index_col = False, names=names)
        datasets[key]['Short'] = 1
        #print('short')
    len_tmp = 0
    

ncyc = 4000
n_init_cyc = {'20_10_1': ncyc,
			  '20_10_2': ncyc,
			  '20_10_3': ncyc,
			  '20_20_1': ncyc,
			  '20_20_2': ncyc,
			  '20_20_3': ncyc,
			  '40_1': ncyc,
			  '40_2': ncyc,
			  '40_3': ncyc}

 
n_len_cyc  = 8000

data_rip = {#'20_10_1_r': datasets['20190828 v20,10-1-reissen.CSV'],
            #'20_10_2_r': datasets['20190828 v20,10-2-reissen.CSV'],
            '20_10_3_r': datasets['20190828 v20-3-reissen.CSV'],
            #'20_20_1_r': datasets['20190828 v20,20-1-reissen.CSV'],
            '20_20_2_r': datasets['20190828 v20,20-2,reissen.CSV'],
            #'20_20_3_r': datasets['20190828 v20,20-3,reissen.CSV'],
            #'40_1_r': datasets['20190828 v40-1-reissen.CSV'],
            '40_2_r': datasets['20190828 v40-2-reissen.CSV'],
            #'40_3_r': datasets['20190828 v40-3-reissen.CSV'],
            #'40_3_r2': datasets['20190828 v40-3-reissen2.CSV']
            }

data_cyc = {'20_10_1': datasets['20190828 v20,10-1.CSV'],
            '20_10_2': datasets['20190828 v20,10-2.CSV'],
            '20_10_3': datasets['20190828 v20-3.CSV'],
            '20_20_1': datasets['20190828 v20,20-1.CSV'],
            '20_20_2': datasets['20190828 v20,20-2.CSV'],
            '20_20_3': datasets['20190828 v20,20-3.CSV'],
            '40_1': datasets['20190828 v40-1.CSV'],
            '40_2': datasets['20190828 v40-2.CSV'],
            '40_3': datasets['20190828 v40-3.CSV'],
            }

#dimensions in the form height x width x length
dimensions_mm = {'20_10_1':[0.580,5,2],
                '20_10_2':[0.580,5,2],
                '20_10_3':[0.580,5,1.5],
                '20_20_1':[0.580,4.5,1.5],
                '20_20_2':[0.580,5,1.5],
                '20_20_3':[0.580,5,2],
                '40_1':[0.580,4,1],
                '40_2':[0.580,5,1],
                '40_3':[0.580,3.5,1],
                '20_10_1_r':[0.580,5,2],
                '20_10_2_r':[0.580,5,2],
                '20_10_3_r':[0.580,5,1.5],
                '20_20_1_r':[0.580,4.5,1.5],
                '20_20_2_r':[0.580,5,1.5],
                '20_20_3_r':[0.580,5,2],
                '40_1_r':[0.580,4,1],
                '40_2_r':[0.580,5,1],
                '40_3_r':[0.580,3.5,1],
                '40_3_r2':[0.580,3.5,1]
                }

idx_max = {     #'20_10_1_r':0,
                #'20_10_2_r':[1950,1970],
                '20_10_3_r':[1950,2000],
                #'20_20_1_r':0,
                '20_20_2_r':[1485,1600],
                #'20_20_3_r':0,
                #'40_1_r':0,
                '40_2_r':[1385,1500],
                #'40_3_r':[1400,1415],
                #'40_3_r2':[1180,1190]
                }

titles = {'20_10_1': 'ULD-V20-ULD, 10 % \n Sample 1',
            '20_10_2': 'Sample 2',
            '20_10_3': 'Sample 3',
            '20_20_1': 'ULD-V20-ULD, 20 % \n Sample 1',
            '20_20_2': 'Sample 2',
            '20_20_3': 'Sample 3',
            '40_1': 'ULD-V40-ULD, 20 % \n Sample 1',
            '40_2': 'Sample 2',
            '40_3': 'Sample 3',
            }


# throw out all data before n_init
for key in data_cyc.keys():
    data_cyc[key] = data_cyc[key][data_cyc[key].Points != 1600]
    data_cyc[key] = data_cyc[key][n_init_cyc[key]:n_init_cyc[key]+n_len_cyc].reset_index()

data_rip['20_10_3_r'] = data_rip['20_10_3_r'].iloc[:-20] 


print(30*'='+'MAXIMUM STRESS'+30*'=')
for key in data_rip.keys():
    print('{} has shape '.format(key), data_rip[key].shape)

print(70*'=')
print(' ')

print(30*'='+'CYCLICAL STRESS'+30*'=')
for key in data_cyc.keys():
    print('{} has shape '.format(key), data_cyc[key].shape)
  #  print(data_cyc[key][795:815])
print(70*'=')
print(' ')


In [ ]:
for key in data_rip.keys(): 
    data_rip[key]['Fs'] = savgol_filter(data_rip[key].Load_2_N,15,1)

    #rename and drop columns
    data_rip[key] = data_rip[key][['Disp_mm','Load_2_N','Fs']]
    data_rip[key].columns = ['D','F','Fs']
    #f = plt.figure(figsize = (14,12))
    #ax = f.add_subplot()
    #data_rip[key].plot(y=['Fs'], ax = ax)
    #plt.plot(data_rip[key].D, s)
    #####spit out plot
    #data_rip[key].plot(y=['D','F','Fs'],figsize = (20,10), title = key)

In [ ]:
#dicts to store the standard deviations for rising and falling flanks for the different cyclical datasets    
stds_rising = {}
stds_falling = {}

for key in data_cyc.keys():
    
    #create a mask for the fft and apply it. after that, shift data upwards to be above zero for causality
    mask = np.ones((data_cyc[key].shape[0]))
    
    '''
    t = np.abs(np.real(np.fft.fft(data_cyc[key].Load_2_N)))[:1000]
    plt.figure(figsize=(15,10))
    plt.plot(t)
    plt.show()
    print(np.argsort(t))
    '''
    mask[0:200] = 0
    mask[-199:] = 0
    mask[201:-200] = 0

    data_cyc[key]['Ff'] = filter_fourier(data_cyc[key].Load_2_N, mask)
    data_cyc[key]['Ff'] = data_cyc[key]['Ff'] - np.min(data_cyc[key]['Ff'])
    
    #create rising and falling flags for all cyclical datasets
    r_tmp = []
    for i in range(data_cyc[key].shape[0]-1):
        if data_cyc[key].Disp_mm.iloc[i] <= data_cyc[key].Disp_mm.iloc[i+1]:
            r_tmp.append(1)
        elif data_cyc[key].Disp_mm.iloc[i]> data_cyc[key].Disp_mm.iloc[i+1]:
            r_tmp.append(-1)
        if i == data_cyc[key].shape[0]-2:
            r_tmp.append(r_tmp[-1])
 
    data_cyc[key]['Rise'] = r_tmp
    stds_rising[key] = np.var(data_cyc[key]['Ff'][data_cyc[key].Rise == 1])**0.5
    stds_falling[key] = np.var(data_cyc[key]['Ff'][data_cyc[key].Rise == -1])**0.5
    
    n_bins = 13
    #bin the displacement data for later use
    data_cyc[key]['D_bin']  = pd.cut(data_cyc[key]['Disp_mm'],n_bins,labels=False)

    #rename and drop columns
    data_cyc[key] = data_cyc[key][['Disp_mm','Load_2_N','Ff','Rise','D_bin']]
    data_cyc[key].columns = ['D','F','Ff','R','Dbin']

In [ ]:
#lets see what we can do here testwise
sigmas = []
epsilons = []
ruptures = {}
%matplotlib inline
for key in data_rip.keys():
    key = key
    d = data_rip[key].copy()
 
    id_max = idx_max[key][0]
    id_min = idx_max[key][1]

    h_gel_mm = dimensions_mm[key][0] 
    b_gel_mm = dimensions_mm[key][1]
    l_gel_mm = dimensions_mm[key][2]

    sigmas.append((d.Fs[id_max])/(b_gel_mm*h_gel_mm))
    epsilons.append(d.D[id_max]/l_gel_mm)
    data = []
    dF = d.Fs[id_max:id_min].max() - d.Fs[id_max:id_min].min()
    data = [1e6*(dF/(b_gel_mm*h_gel_mm)), (d.D[id_max]/l_gel_mm)]
    ruptures[key] = data
    
    #f = plt.figure(figsize=(20,15))
    #ax = f.add_subplot()
    d.D  = d.D / l_gel_mm
    #d.iloc[300:].plot(y=['F','Fs'],x='D', ax = ax)

In [ ]:
ruptures

In [ ]:
#saving rupture data to csv file
#dfrupt = pd.DataFrame.from_dict(ruptures,orient='index').T
#dfrupt.index = ['Stress at rupture in N/m^2','strain at rupture']
#dfrupt
#dfrupt.to_csv('rupture_data.csv')
#ylims[0]

In [ ]:
import copy

mpl.rcParams['font.size'] = 8
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['lines.markersize'] = 2
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['figure.titlesize'] = 'small'

sigma_string = r'$\sigma$  /  $\left[\dfrac{N}{m^2}\right]$'
eplsilon_string = r'$\varepsilon$  /  $\left[ \% \right]$'
y_modulus = {}

fig = plt.figure(constrained_layout=True, figsize=(12,12))
spec = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)
nc = 0
nr = 0


# Set scale_axes = True, if plot axes shoudl be scaled to same maximum value fgür each gel type
scale_axes = True
ylims = {'0':30000,'1':35000,'2':52000}
xlims = 22



#iterate through columns
for key in data_cyc.keys():
    if nr == 3:
        nc +=1
        nr = 0
 
    ax = fig.add_subplot(spec[nr,nc])
    d = data_cyc[key]
    
    # get geometric data of gels for sigma and epsilon
    
    h_gel_mm = dimensions_mm[key][0] 
    b_gel_mm = dimensions_mm[key][1]
    l_gel_mm = dimensions_mm[key][2]
    
    #plot 4 curves, each consisting of 2000 points, to investigate fatigue
    
    for k in range(1,5):
       
        dtot = data_cyc[key].iloc[(k-1)*2000:k*2000].copy()
        mask = np.ones((dtot.shape[0]))
        mask[0:50] = 0
        mask[-49:] = 0

        mask[51:-50] = 0

        dtot['Ff'] = filter_fourier(dtot.F, mask)
        dtot['Ff'] = dtot['Ff'] - np.min(dtot['Ff'])
        dtot['Eps'] = 100 * (dtot.D / l_gel_mm)
        dtot['Sigma'] = 1e6 * (dtot.Ff / (h_gel_mm*b_gel_mm))
        
        dtot.plot(x='Eps',y='Sigma', ax = ax, color = (.7**k, .7**k, .95**k), label = 'cycles '  + str(int((k-1)*2000/40+1))+ '-' +str(int(k*2000/40)))
                
        df = np.max(dtot['Ff']) - np.min(dtot['Ff'])
        dsigma = df/(b_gel_mm*h_gel_mm)
        
        epsilon = (np.max(dtot['D']) - np.min(dtot['D']))/l_gel_mm
        #save the youngs modulus for each cycle group
        y_modulus[key+'_'+str(k)] = 1e6*dsigma/epsilon
        
        # remove legends from all subplots
        ax.get_legend().remove()
        
   # ax.set_ylim([-0.01,0.10])
    ax.set_title(titles[key])
    ax.set_xlabel(eplsilon_string)
    ax.set_ylabel(sigma_string)
    
    if scale_axes:
        ax.set_ylim([-0.1*ylims[str(nc)],ylims[str(nc)]])
        ax.set_xlim([-0.1*xlims,xlims])
        ax.grid(True)
    nr += 1
    
#get handels and labels for the last axes to create a single legend above all subplots
handles, labels = ax.get_legend_handles_labels()
# copy the handles

handles = [copy.copy(ha) for ha in handles]

# set the linewidths to the copies
[ha.set_linewidth(7) for ha in handles]

fig.legend(handles, 
           labels, 
           ncol=4, 
           borderaxespad=0., 
           fontsize = 14,
           mode="expand",
           #loc='upper center', 
           bbox_to_anchor=(0., .95, 1., .102))


#fig.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=2, mode="expand", borderaxespad=0.)

#plt.tight_layout()
    
nc = 0    

fig.savefig('Figure3_DMA.svg',format='svg',bbox_inches = 'tight')
fig.savefig('Figure3_DMA.png',format='png',bbox_inches = 'tight',dpi=200)

In [ ]:
#y_modulus
dfyoung = pd.DataFrame.from_dict(y_modulus,orient='index').reset_index()
dfyoung.columns=['Gel','E']
dfyoung.Gel = dfyoung.Gel.str.slice(0,-2)
dfyoung_grouped = dfyoung.groupby('Gel').agg({'E': [np.mean,np.std]}).round(2)
dfyoung_grouped.columns = ['E_mu','E_std']
dfyoung_grouped.to_csv('y_mod_grouped.csv')

In [ ]:
f = plt.figure(figsize = (10,10))
ax = f.add_subplot(1,1,1)
data_cyc['20_10_1'][:500].plot(y='Ff', ax = ax)

In [ ]:
#sub = pd.DataFrame.from_dict(y_modulus,orient='index')
#sub
#sub.to_csv('y_mod.csv')

In [ ]:
#ruptures
#sub_rup = pd.DataFrame.from_dict(ruptures, orient='index')
#sub_rup.to_csv('rup.csv')

In [ ]:
##=========================================================##
##HERE COMES THE DMA DATA FROM THE 22.12.2020##
data_dir = 'Messungen/DMA 22.12.2020/'
files = getFileContent(data_dir)

datasets = {}

#renaming those absolutely horribly chosen column names
names = ['Points','Elapsed_Time_Sec','Scan_Time_Sec','Disp_mm','Load_2_N','E12_%','E22_%','Axial_cmd_V','Axial_err_%','Unnamed']
k=0
for i in files.keys():
    df = pd.read_csv(files[i],header = 2, index_col = False,names=names)
    if (df.shape[0]<10000) | (files[i][-5]  != '1'):
        continue
    df = df.iloc[4000:12000].Load_2_N.values
    print(i)
    
    #obtain hightes spectral energy but kill dc component at index zero
    spectrum = df.copy()
    spectrum = np.fft.fft(spectrum)#.reshape(8000,1)
    spectrum[0] = 0
    figsize = (14,12)
    plt.figure(figsize=figsize)
    plt.plot(spectrum)
    plt.title(i)
    
    print(np.argmax(spectrum[:4000]))
    f_accept = np.argmax(np.abs(np.real(spectrum[:4000])))
    bw = 0
    
    mask = np.zeros((8000))
    mask[f_accept-bw:f_accept+bw+1] = 1
    mask[-f_accept-bw:-f_accept+bw+1] = 1
    
    result = spectrum*mask
    result = np.real(np.fft.ifft(result))
    
    plt.figure(figsize=figsize)
    plt.plot(result[:1000],alpha=0.45, label=i)
    plt.plot(df[:1000],alpha=0.45)
    plt.legend()
    k+=1
    if k==20:
        break


In [ ]:
a =
b = 
c = 

anregungsfrequenz =

